In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
res = pd.DataFrame(columns=['chr', 'method', 'load_time', 'load_peak_memory', 'dot_product_time', 'dot_product_peak_memory'])

df = pd.read_csv('/Users/ambershen/Desktop/linARG/dx_analysis/figures/4a/data/tmp_dot_product_time_all.csv')

for method in ['grg', 'linarg', 'genotypes']:
    for chrom in [1, 11, 21]:        
        df_filt = df[(df.chr==chrom) & (df['method'].str.startswith(method))]
        
        load_time = np.sum(df_filt['load_time (s)'])
        load_peak_memory = np.sum(df_filt['load_memory (MB)'])
        dot_product_time = np.sum(df_filt['dot_product_time (s)'])
        dot_product_peak_memory = np.sum(df_filt['dot_product_memory (MB)'])
        
        res.loc[res.shape[0]] = [chrom, method, load_time, load_peak_memory, dot_product_time, dot_product_peak_memory]

In [5]:
res

chr     method    load_time  load_peak_memory  dot_product_time  \
0    1        grg   233.669696      35081.987500         17.056883   
1   11        grg   160.091072      25564.803516         10.758467   
2   21        grg    40.531252       7523.349609          3.186418   
3    1     linarg    37.883671      23714.703906          6.366982   
4   11     linarg    22.299925      14149.546875          3.432288   
5   21     linarg     6.928738       4347.868750          0.998868   
6    1  genotypes  1151.191025     925060.016016        252.991659   
7   11  genotypes   733.206870     593102.444141        163.753553   
8   21  genotypes   204.927594     165452.731641         46.066399   

   dot_product_peak_memory  
0             2.149899e+04  
1             1.522511e+04  
2             4.532375e+03  
3             2.090814e+04  
4             1.242435e+04  
5             3.782043e+03  
6             1.084373e+06  
7             6.960093e+05  
8             1.943936e+05

In [6]:
for chrom in [1, 11, 21]:
    res_filt = res[res.chr==chrom]
    grg_lt = np.sum(res_filt[res_filt.method=='grg'].load_time)
    linarg_lt = np.sum(res_filt[res_filt.method=='linarg'].load_time)
    grg_dpt = np.sum(res_filt[res_filt.method=='grg'].dot_product_time)
    linarg_dpt = np.sum(res_filt[res_filt.method=='linarg'].dot_product_time)
    scipy_lt = np.sum(res_filt[res_filt.method=='genotypes'].load_time)
    scipy_dpt = np.sum(res_filt[res_filt.method=='genotypes'].dot_product_time)
    
    print(chrom)
    print(f'grg lt ratio: {grg_lt / linarg_lt}')
    print(f'scipy lt ratio: {scipy_lt / linarg_lt}')
    
    print(f'grg dpt ratio: {grg_dpt / linarg_dpt}')
    print(f'scipy dpt ratio: {scipy_dpt / linarg_dpt}')
    
    print(f'grg total ratio: {(grg_lt+grg_dpt) / (linarg_lt+linarg_dpt)}')
    print(f'scipy total ratio: {(scipy_lt+scipy_dpt) / (linarg_lt+linarg_dpt)}')
    

1
grg lt ratio: 6.168084854033242
scipy lt ratio: 30.387525838312026
grg dpt ratio: 2.678958732474302
scipy dpt ratio: 39.7349396428581
grg total ratio: 5.6660537470443835
scipy total ratio: 31.732473637929687
11
grg lt ratio: 7.178995952492574
scipy lt ratio: 32.87934223080011
grg dpt ratio: 3.1344888342518105
scipy dpt ratio: 47.70974045758829
grg total ratio: 6.639519930179266
scipy total ratio: 34.85749284614383
21
grg lt ratio: 5.84973098147365
scipy lt ratio: 29.576468098996145
grg dpt ratio: 3.1900285387973457
scipy dpt ratio: 46.118596896836536
grg total ratio: 5.514611891197347
scipy total ratio: 31.66075507223647


In [7]:
res_clean = pd.DataFrame(columns=['method', 'load_time', 'load_peak_memory', 'dot_product_time', 'dot_product_peak_memory'])

for method in ['genotypes', 'grg', 'linarg']:    
    res_clean.loc[res_clean.shape[0]] = [method] + list(res[res.method==method][['load_time', 'load_peak_memory', 'dot_product_time', 'dot_product_peak_memory']].sum(axis=0))

res_clean

method    load_time  load_peak_memory  dot_product_time  \
0  genotypes  2089.325489      1.683615e+06        462.811610   
1        grg   434.292021      6.817014e+04         31.001768   
2     linarg    67.112334      4.221212e+04         10.798138   

   dot_product_peak_memory  
0             1.974776e+06  
1             4.125647e+04  
2             3.711453e+04

In [9]:
methods = ['xsi', 'plink2']
times = {x: 0 for x in methods}
for method in methods:
    for chrom in [1, 11, 21]:
        df = pd.read_csv(f'/Users/ambershen/Desktop/linARG/dx_analysis/figures/4a/data/dot_product_{method}_chr{chrom}.csv')
        times[method] += np.mean(df.wall_time_seconds)
times

{'xsi': 1835.714, 'plink2': 276.09999999999997}

In [15]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

methods = ['scipy', 'xsi', 'grg', 'plink2', 'linarg']

load_methods = ['scipy', 'grg', 'linarg']
load_time = {x: y for x,y in zip(load_methods, res_clean.load_time)}
dot_product_time = {x: y for x,y in zip(load_methods, res_clean.dot_product_time)}

y = np.arange(len(methods))

colors = ['#93c47d', '#f6b26b', '#56C1FF', '#E9B824', '#758acfff']

hatches = ['///', ''] 

fig, ax = plt.subplots(figsize=(10, 5))

# Plot each bar segment individually for full control
for i in range(len(methods)):
    print(methods[i])
    
    if methods[i] in load_methods:
        # First subcategory
        ax.barh(y[i], load_time[methods[i]], color=colors[i], hatch=hatches[0], edgecolor='black', label='Load Time' if i == 0 else "")
        
        # Second subcategory stacked on top
        ax.barh(y[i], dot_product_time[methods[i]], 
                left=load_time[methods[i]], 
                color=colors[i], 
                hatch=hatches[1], 
                edgecolor='black', 
                label='Dot Product Time' if i == 0 else "")
    else:
            ax.barh(y[i], times[methods[i]], 
            color=colors[i], 
            hatch='', 
            edgecolor='black', 
            label='Total time' if i == 0 else "")
        

# Set y-axis labels and reverse order
ax.set_yticks(y)
ax.set_yticklabels(['SciPy', 'XSI', 'GRG', 'Plink2', 'Linear ARG'], fontsize=12)
ax.invert_yaxis()

# Labels and title
ax.set_xlabel('Time to compute dot product (s)', fontsize=12)

load_method_total_times = np.array(list(load_time.values())) + np.array(list(dot_product_time.values()))
other_method_total_times = np.array(list(times.values()))
all_times = sorted(np.concatenate((load_method_total_times, other_method_total_times)), reverse=True)

linarg_time = all_times[-1]
for i in range(len(all_times)-1):  # Only for scipy and grg
    speedup = all_times[i] / linarg_time
    ax.text(all_times[i] + 0.05, y[i], f"  {speedup:.1f}×", va='center', fontsize=12)

legend_handles = [
    mpatches.Patch(facecolor='white', edgecolor='black', hatch=hatches[0], label='Load Time'),
]
ax.legend(handles=legend_handles, fontsize=12, loc='lower right')

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.tight_layout()
plt.show()


scipy
xsi
grg
plink2
linarg


In [ ]:
load_method_total_times = np.array(list(load_time.values())) + np.array(list(dot_product_time.values()))
other_method_total_times = np.array(list(times.values()))
all_times = np.sort(np.concatenate((load_method_total_times, other_method_total_times)), reverse=True)

In [32]:
all_times

array([   77.91047192,   465.29378893,  1835.714     ,  2552.13709892,
       12667.        ])